# Fortinet VPN Logs Analysis

This notebook helps to deal with VPN logs from Fortinet firewalls.
Although data conversion can be used for all types of "key-value" logs, data processing will not work with another type of log due to the column names.

## Logs conversion to csv

Use of multiprocessing to convert Fortinet firewall log files with non csv format to csv.

In [1]:
def read_files_folder(path):
    from os import walk
    files = []
    for (dirpath, dirnames, filenames) in walk(path):
        for filename in filenames:
            if filename.endswith("log"):
                file_path = dirpath + "\\" + filename
                files.append(file_path)
    return files

In [12]:
def write_file_functions(output_path=r"C:\Logs_VPN_Events\logs_csv"):
    f = open(r"process_file.py", "w+")

    file_content = f"""def load_log(log_string_list):
    import re
    import pandas as pd
    pattern = re.compile('(\w+)=(?:"([^"]*)"|(\S*))')

    lines_s = pd.Series(log_string_list)
    data = lines_s.str.extractall(pattern)
    data.index = data.index.get_level_values(0)
    data[1] = data[1].fillna(data[2])
    return data.pivot(index=None, columns=0, values=1)
    
    
def process_file(file):
    import pandas as pd
    from os import path

    output_folder = r'{output_path}'
    first = True
    with open(file, "r") as f:
        if first:
            log_df = load_log(f.readlines())
            first = False
        else:
            t_df = load_log(f.readlines())
            log_df = log_df.append(t_df)
            del t_df
    file_name = path.basename(file)
    file_path = output_folder + '\\\\' + file_name + '.csv'
    log_df.to_csv(file_path)

    del log_df"""

    f.write(file_content)
    f.close()

In [13]:
write_file_functions()

In [2]:
%%writefile process_file.py

def load_log(log_string_list):
    import re
    import pandas as pd
    pattern = re.compile('(\w+)=(?:"([^"]*)"|(\S*))')
    
    lines_s = pd.Series(log_string_list)
    data = lines_s.str.extractall(pattern)
    data.index = data.index.get_level_values(0)
    data[1] = data[1].fillna(data[2])
    return data.pivot(index=None, columns=0, values=1)

def process_file(file):
    import pandas as pd
    from os import path
    
    output_folder = r"C:\Logs_VPN_Events\logs_csv"
    first = True
    with open(file, "r") as f:
        if first:
            log_df = load_log(f.readlines())
            first = False
        else:
            t_df = load_log(f.readlines())
            log_df = log_df.append(t_df)
            del t_df
    file_name = path.basename(file)
    file_path = output_folder + "\\" + file_name + ".csv"
    log_df.to_csv(file_path)
    
    del log_df

Overwriting process_file.py


In [14]:
import process_file

In [3]:
%%time
from multiprocessing import Pool
import process_file

if __name__ ==  '__main__': 
    num_processors = 12
    p=Pool(processes = num_processors)
    logs_folder = r"C:\Logs_VPN_Events"

    files = read_files_folder(logs_folder)

    p.map(process_file.process_file, files)
    
    p.close()
    p.join()

Wall time: 35min 10s


## Dealing with different columns in files

In [36]:
from sklearn import preprocessing

In [5]:
import glob
import d6tstack.combine_csv

cfg_fnames = list(glob.glob(r"C:\logs_csv\*.csv"))
c = d6tstack.combine_csv.CombinerCSV(cfg_fnames)

# check columns
print('all equal',c.is_all_equal())
print('')
d = c.is_column_present()

sniffing columns ok
all equal False



In [ ]:
fnames = d6tstack.combine_csv.CombinerCSV(cfg_fnames).to_csv_align(output_dir=r'C:\logs_csv\fixed')

## Reading log files with Dask from disk 

In [1]:
import dask.dataframe as dd
import pandas as pd

In [2]:
%%time
df_dask = dd.read_csv(r'C:\logs_vpn_out_fix\*.csv',dtype={'error_num': 'object',
        'group': 'object','locport': 'object','peer_notif': 'object','reason': 'object','remport': 'object',
        'seq': 'object','spi': 'object','tunnelip': 'object','tunneltype': 'object','user': 'object',
        'desc': 'object','espauth': 'object','esptransform': 'object','in_spi': 'object','out_spi': 'object',
        'rcvdbyte': 'float','stage': 'object','tunnelid': 'object','Unnamed: 0': 'object','assignip': 'object',
        'dst_host': 'object','duration': 'float','eventtime': 'object','logid': 'object','nextstat': 'object',
        'phase2_name': 'object','sentbyte': 'float','exch': 'object','version': 'object'})

Wall time: 64.1 ms


## Researching

# Useful functions

In [3]:
# MaxMind ASN
import geoip2.database
def get_asn(ip):
    try:
        x = reader.asn(ip)
    except:
        return pd.np.nan
    try:
        return x.autonomous_system_organization if x else pd.np.nan
    except KeyError:
        return pd.np.nan

# MaxMind Country
def get_country(ip):
    try:
        x = reader.country(ip)
    except:
        return pd.np.nan
    try:
        return x.country.name if x else pd.np.nan
    except KeyError:
        return pd.np.nan

In [4]:
# Tor Nodes List
import requests
nodes = requests.request(method='GET', url="https://www.dan.me.uk/torlist/") #30 minutes limit
nodes_list = nodes.text.split("\n")

def is_tor_node(ip):
    if ip in nodes_list:
        return True
    else:
        return False

In [5]:
def abuseipdb_req(ip):
    import json
    try:
        key = <api-key>
        url = 'https://api.abuseipdb.com/api/v2/check'
        querystring = {
            'ipAddress': ip,
            'maxAgeInDays': '90'
        }
        headers = {
            'Accept': 'application/json',
            'Key': key
        }
        response = requests.request(method='GET', url=url, headers=headers, params=querystring)
        decodedResponse = json.loads(response.text)
    except e:
        print(e)
        return pd.np.nan
    try:
        return decodedResponse["data"]["abuseConfidenceScore"] if decodedResponse else pd.np.nan
    except KeyError:
        return pd.np.nan

In [6]:
# Convert bytes to human readable
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

## Searching for information exfiltration

In [ ]:
%%time
df_exf = df_dask[["remip", "tunnelid", "sentbyte"]].groupby(["remip","tunnelid"]).max().compute()

In [114]:
df_exf.reset_index(inplace=True)

In [115]:
df_exf["h_sentbyte"] = df_exf.sentbyte.apply(sizeof_fmt)

In [122]:
df_ext_10 = df_exf.nlargest(10, "sentbyte").sort_values(by="sentbyte", ascending=False).reset_index()

In [124]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'

In [127]:
fig = px.bar(df_ext_10.iloc[2:8], y="remip", x="sentbyte", title='Destiantion IPs by data sent', orientation='h')
fig.show()

### Data mining

In [129]:
%%time
## Add ASN
with geoip2.database.Reader(r'C:\maxmind\GeoLite2-ASN.mmdb') as reader:
    unique_ips = df_exf['remip'].unique()
    unique_ips = pd.Series(unique_ips, index = unique_ips)
    df_exf['ASN'] = df_exf['remip'].map(unique_ips.apply(get_asn))

Wall time: 34.1 ms


In [132]:
df_exf.nlargest(10, "sentbyte")

,index,remip,tunnelid,sentbyte,h_sentbyte,ASN
495,495,103.6.139.246,2973221928.0,3.470560e+12,3.2TiB,HKBN Enterprise Solutions HK Limited
1,1,103.6.139.246,3054928288.0,1.646262e+12,1.5TiB,HKBN Enterprise Solutions HK Limited
164,164,191.101.209.126,1826793719.0,3.506824e+10,32.7GiB,GTT Communications Inc.
214,214,191.101.209.72,1826793700.0,2.084387e+10,19.4GiB,GTT Communications Inc.
291,291,191.101.209.72,1826793661.0,1.833587e+10,17.1GiB,GTT Communications Inc.
361,361,115.134.131.154,3068782335.0,1.560168e+10,14.5GiB,"TM Net, Internet Service Provider"
293,293,171.99.130.158,3069050613.0,8.586856e+09,8.0GiB,"TRUE INTERNET Co.,Ltd."
37,37,210.61.150.144,3067193495.0,7.414605e+09,6.9GiB,Data Communication Business Group
32,32,202.188.229.70,3027033592.0,5.857543e+09,5.5GiB,"TM Net, Internet Service Provider"
13,13,122.103.241.185,3069552181.0,4.883923e+09,4.5GiB,ARTERIA Networks Corporation


In [133]:
%%time
## Add country
with geoip2.database.Reader(r'C:\maxmind\GeoLite2-Country.mmdb') as reader:
    unique_ips = df_exf['remip'].unique()
    unique_ips = pd.Series(unique_ips, index = unique_ips)
    df_exf['remip_country'] = df_exf['remip'].map(unique_ips.apply(get_country))

Wall time: 44.9 ms


In [ ]:
df_exf.nlargest(10, "sentbyte")

In [157]:
df_ext_map = df_exf.sort_values(by="sentbyte", ascending=False).iloc[2:]

In [158]:
df_ext_map = df_ext_map[["sentbyte", "remip_country"]].groupby("remip_country").sum().reset_index()

In [171]:
import plotly as py
import plotly.graph_objs as go
import pandas as pd

data = dict (
    type = 'choropleth',
    locations = df_ext_map['remip_country'],
    locationmode='country names',
    colorscale = 'blugrn',
    z=df_ext_map['sentbyte'])

map = go.Figure(data=[data])
py.offline.plot(map)

'temp-plot.html'

### Search for Tor nodes communications

In [174]:
%%time
unique_remip = df_exf["remip"].unique()
unique_remip = pd.Series(unique_remip, index = unique_remip)
df_exf['is_tor_node'] = df_exf['remip'].map(unique_remip.apply(is_tor_node))

Wall time: 18 ms


In [176]:
df_exf[df_exf["is_tor_node"]]

,index,remip,tunnelid,sentbyte,h_sentbyte,ASN,remip_country,is_tor_node
189,189,185.220.100.247,0.0,NaN,nanYiB,F3 Netze e.V.,Germany,True
196,196,45.129.56.200,0.0,NaN,nanYiB,31173 Services AB,Denmark,True
210,210,185.220.100.244,0.0,NaN,nanYiB,F3 Netze e.V.,Germany,True
211,211,185.220.101.129,0.0,NaN,nanYiB,Cia Triad Security LLC,Germany,True


In [ ]:
df_exf.nlargest(10, "sentbyte")

### AbuseIPDB score

In [181]:
%%time
unique_ips = df_exf["remip"].unique()
unique_ips = pd.Series(unique_ips, index = top10)
df_exf['remip_abuse'] = df_exf['remip'].map(unique_ips.apply(abuseipdb_req))

Wall time: 2min 22s


In [182]:
df_exf.nlargest(10, "remip_abuse")

,index,remip,tunnelid,sentbyte,h_sentbyte,ASN,remip_country,is_tor_node,remip_abuse
16,16,162.142.125.39,0.0,NaN,nanYiB,CENSYS-ARIN-01,United States,False,100
31,31,192.241.210.30,0.0,NaN,nanYiB,DIGITALOCEAN-ASN,United States,False,100
49,49,167.248.133.53,0.0,NaN,nanYiB,CENSYS-ARIN-03,United States,False,100
57,57,111.7.96.166,0.0,NaN,nanYiB,Guangdong Mobile Communication Co.Ltd.,China,False,100
62,62,184.105.139.69,0.0,NaN,nanYiB,HURRICANE,United States,False,100
64,64,65.49.20.69,0.0,NaN,nanYiB,HURRICANE,United States,False,100
95,95,74.120.14.40,0.0,NaN,nanYiB,CENSYS-ARIN-02,United States,False,100
114,114,74.120.14.39,0.0,NaN,nanYiB,CENSYS-ARIN-02,United States,False,100
138,138,162.142.125.128,0.0,NaN,nanYiB,CENSYS-ARIN-01,United States,False,100
142,142,184.105.247.195,0.0,NaN,nanYiB,HURRICANE,United States,False,100


## Tor Nodes communications

In [92]:
%%time
df_tor = df[["srcip","dstip"]].compute()

Wall time: 24.2 s


In [103]:
%%time
unique_ips_src = df_tor['srcip'].unique()
unique_ips_dst = df_tor['dstip'].unique()

unique_ips_src = pd.Series(unique_ips_src, index = unique_ips_src)
unique_ips_dst = pd.Series(unique_ips_dst, index = unique_ips_dst)

df_tor['is_tor_node_src'] = df_tor['srcip'].map(unique_ips_src.apply(is_tor_node))
df_tor['is_tor_node_dst'] = df_tor['dstip'].map(unique_ips_dst.apply(is_tor_node))

Wall time: 14.5 s


In [104]:
df_tor[df_tor["is_tor_node_dst"]]["dstip"].value_counts()

51.15.47.139      157
46.105.55.153       8
83.135.89.183       6
82.65.125.114       6
185.21.216.195      6
188.126.94.37       3
188.40.142.18       1
Name: dstip, dtype: int64

In [105]:
df_tor[df_tor["is_tor_node_src"]]["srcip"].value_counts()

45.129.56.200      141
185.220.100.240     90
171.25.193.78       85
185.220.100.252     70
185.220.100.243     55
185.220.101.144     40
185.220.101.6       33
205.185.116.3       26
142.44.246.156      22
185.220.100.250     20
84.209.139.0        19
162.247.74.204      19
199.195.251.84      16
37.187.196.70       12
185.220.100.246     11
163.172.213.212     11
185.220.102.251     11
185.220.100.242     11
198.144.120.177     10
107.189.10.42       10
209.141.55.26       10
77.247.181.163      10
185.220.101.7       10
185.220.101.136      9
199.195.250.77       9
185.83.214.69        9
51.210.80.127        9
185.220.102.4        8
176.10.104.240       8
5.2.69.50            8
185.220.101.142      8
185.220.100.255      8
185.220.100.247      8
18.27.197.252        8
89.163.154.91        7
89.163.249.244       4
185.220.101.146      3
64.113.32.29         2
185.220.101.137      1
185.220.101.1        1
Name: srcip, dtype: int64

### Data mining

In [4]:
%%time
df = df_dask[~df_dask["tunnelid"].isnull()][["remip", "duration", "tunnelid","sentbyte"]].compute()

Wall time: 1min 20s


In [5]:
df_work = df.copy()

In [6]:
import numpy as np

In [7]:
df_final = df_work.remip.unique()
df_final = pd.Series(df_final).to_frame().rename(columns={0:"remip"})

In [8]:
duration_t = df_work[["remip", "duration", "tunnelid"]].groupby(["remip", "tunnelid"]).max().dropna().reset_index()

In [9]:
df_final["n_tunnels"] = df_final["remip"].map(duration_t[["remip", "duration"]].groupby("remip").count()["duration"])
df_final["n_connections"] = df_final["remip"].map(df_work.value_counts("remip"))
df_final["duration_sum"] = df_final["remip"].map(duration_t[["remip", "duration"]].groupby("remip").sum()["duration"])
df_final["duration_min"] = df_final["remip"].map(duration_t[["remip", "duration"]].groupby("remip").agg(np.min)["duration"])
df_final["duration_max"] = df_final["remip"].map(duration_t[["remip", "duration"]].groupby("remip").agg(np.max)["duration"])
df_final["duration_mean"] = df_final["remip"].map(duration_t[["remip", "duration"]].groupby("remip").agg(np.mean)["duration"])
df_final["duration_var"] = df_final["remip"].map(duration_t[["remip", "duration"]].groupby("remip").agg(np.var)["duration"])

In [10]:
sentby_t = df_work[~df_work["tunnelid"].isnull()][["remip", "sentbyte", "tunnelid"]].groupby(["remip", "tunnelid"]).max().dropna().reset_index()

In [11]:
df_final["sentbyte_sum"] = df_final["remip"].map(sentby_t[["remip", "sentbyte"]].groupby("remip").sum()["sentbyte"])
df_final["sentbyte_min"] = df_final["remip"].map(sentby_t[["remip", "sentbyte"]].groupby("remip").agg(np.min)["sentbyte"])
df_final["sentbyte_max"] = df_final["remip"].map(sentby_t[["remip", "sentbyte"]].groupby("remip").agg(np.max)["sentbyte"])
df_final["sentbyte_mean"] = df_final["remip"].map(sentby_t[["remip", "sentbyte"]].groupby("remip").agg(np.mean)["sentbyte"])
df_final["sentbyte_var"] = df_final["remip"].map(sentby_t[["remip", "sentbyte"]].groupby("remip").agg(np.var)["sentbyte"])

In [12]:
df_final = df_final[~df_final["n_tunnels"].isnull()]

In [13]:
df_final.sort_values(by="n_tunnels", ascending=False)

,remip,n_tunnels,n_connections,duration_sum,duration_min,duration_max,duration_mean,duration_var,sentbyte_sum,sentbyte_min,sentbyte_max,sentbyte_mean,sentbyte_var
88,210.22.132.214,29.0,550,106014.0,114.0,15499.0,3655.655172,1.586899e+07,1.005893e+10,0.0,1.707831e+09,3.468595e+08,2.064729e+17
5,211.36.1.126,26.0,1361,744569.0,24599.0,28801.0,28637.269231,6.784200e+05,1.212432e+09,2582718.0,1.259659e+08,4.663198e+07,1.850490e+15
2,115.134.131.154,18.0,1717,3128632.0,1164.0,2509427.0,173812.888889,3.523311e+11,1.790880e+10,167265.0,1.560168e+10,9.949331e+08,1.357241e+19
6,113.161.106.18,13.0,583,297653.0,215.0,56986.0,22896.384615,4.215194e+08,4.110226e+08,25678.0,1.150723e+08,3.161712e+07,1.850948e+15
3,180.168.169.90,12.0,526,166454.0,117.0,36956.0,13871.166667,2.085070e+08,5.226736e+07,23420.0,1.299919e+07,4.355613e+06,2.089468e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,223.24.190.29,1.0,55,26906.0,26906.0,26906.0,26906.000000,NaN,8.000106e+06,8000106.0,8.000106e+06,8.000106e+06,NaN
101,58.153.80.60,1.0,14,367.0,367.0,367.0,367.000000,NaN,8.554023e+06,8554023.0,8.554023e+06,8.554023e+06,NaN
103,49.130.126.96,1.0,33,13010.0,13010.0,13010.0,13010.000000,NaN,3.473580e+07,34735804.0,3.473580e+07,3.473580e+07,NaN
114,10.14.141.102,1.0,52,15298.0,15298.0,15298.0,15298.000000,NaN,8.645677e+06,8645677.0,8.645677e+06,8.645677e+06,NaN


### PCA to reduce components to 2

In [23]:
# Replace NaN with 0 and apply PCA. No Label Encoder
def apply_pca2(df_int):
    from sklearn.decomposition import PCA
    df_int.replace(np.nan, "0", inplace=True)
    
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(df_int)

    principalDf = pd.DataFrame(data=principalComponents, columns=['x', 'y'])
    
    return principalDf

In [24]:
df_pca = df_final[["n_tunnels","n_connections", "duration_sum","sentbyte_sum"]]

In [25]:
df_pca_final = apply_pca2(df_pca)

c:\python38\lib\site-packages\pandas\core\frame.py:5233: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [26]:
remip = df_final["remip"].reset_index()
dfjoin = df_pca_final.join(remip)

In [32]:
dfjoin

,x,y,remip
0,-5.662833e+10,-3.446632e+05,10.14.141.158
1,-5.663200e+10,-3.626998e+05,192.168.0.16
2,-3.875444e+10,2.700874e+06,115.134.131.154
3,-5.661097e+10,-2.053365e+05,180.168.169.90
4,-4.718642e+10,2.614039e+06,171.99.130.158
...,...,...,...
88,-5.665053e+10,-3.699487e+05,10.14.141.119
89,-5.665975e+10,-3.683208e+05,181.215.176.250
90,-5.665127e+10,-3.453214e+05,223.24.94.33
91,-5.666324e+10,-3.711788e+05,222.125.64.74


In [27]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe' # or 'notebook' or 'colab'

### PCA "n_tunnels","n_connections", "duration_sum","sentbyte_sum"

In [28]:
fig = px.scatter(dfjoin, x="x", y="y", color='remip', title='Scatter VPN connections PCA "n_tunnels","n_connections", "duration_sum","sentbyte_sum"',
                 color_discrete_sequence=["blue"],
             color_discrete_map={
                "191.101.209.126": "red",
                 "191.101.209.72":"red"
             })
fig.show()

### Without PCA duration, sentbyte

In [29]:
fig = px.scatter(df_final, x="duration_sum", y="n_connections", color='remip', title='Scatter VPN connections without PCA Connections VS Duration',
                 color_discrete_sequence=["blue"],
             color_discrete_map={
                "191.101.209.126": "red",
                 "191.101.209.72":"red"
             })
fig.show()

In [17]:
fig = px.scatter(df_final, x="duration_sum", y="sentbyte_sum", color='remip', title='Scatter VPN connections without PCA, Duration VS Sentbyte',
                 color_discrete_sequence=["blue"],
             color_discrete_map={
                "191.101.209.126": "red",
                 "191.101.209.72":"red"
             })
fig.show()

In [18]:
fig = px.scatter(df_final, x="n_connections", y="sentbyte_sum", color='remip', title='Scatter VPN connections without PCA Connections VS Sentbyte',
                 color_discrete_sequence=["blue"],
             color_discrete_map={
                "191.101.209.126": "red",
                 "191.101.209.72":"red"
             })
fig.show()